In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

df = pd.read_json('../data/5.final_ingredient_data.json')

In [ ]:
df

In [ ]:
uniq_external_id = df['tags'].apply(lambda tags: [tag["externalId"] for tag in tags if tag["externalId"] != 'main-dish-recipes']).tolist()

In [ ]:
uniq_external_id = list(set(sum(uniq_external_id, [])))

In [ ]:
uniq_external_id

In [ ]:
df['tags2'] = df['tags'].apply(lambda tags: [tag['externalId'] for tag in tags])

In [ ]:
df[['tags2']]

In [ ]:
def is_external_id_in_list(external_ids, check_external_id):
  if(check_external_id in external_ids):
    return 1
  else:
    return 0

for external_id in uniq_external_id:
  df[external_id] = df['tags2'].apply(lambda x : is_external_id_in_list(x,external_id))

In [ ]:
df

In [ ]:
import sys
from pathlib import Path

# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0])

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from dictionary.ingredient_dict import ingredient_dict
from dictionary.unit_dict import measure, amount_for_uncount_unit

In [ ]:
def set_amount(ingredient):
  if ingredient['unit'] in measure:
    return ingredient['amount']*measure[ingredient['unit']] 
  elif ingredient['unit'] in amount_for_uncount_unit[ingredient['ingredientName']]:
    return ingredient['amount']*amount_for_uncount_unit[ingredient['ingredientName']][ingredient['unit']]
  return ingredient['amount']

In [ ]:
def is_ingredients_in_list(ingredients, ingre_name):
  for ingredient in ingredients:
    if ingredient['ingredientName'] == ingre_name:
      return set_amount(ingredient)
  return 0

In [ ]:
# is_ingredients_in_list([{'ingredientName': 'วุ้นเส้น', 'amount': 1, 'unit': 'ซอง'},
#  {'ingredientName': 'หมูกรอบ', 'amount': 1, 'unit': 'cup'},
#  {'ingredientName': 'ใบกะเพรา', 'amount': 1, 'unit': 'cup'},
#  {'ingredientName': 'พริก', 'amount': 4, 'unit': 'เม็ด'},
#  {'ingredientName': 'กระเทียม', 'amount': 4, 'unit': 'lobe'},
#  {'ingredientName': 'ซีอิ๊วขาว', 'amount': 0.5, 'unit': 'table_spoon'},
#  {'ingredientName': 'น้ำตาล', 'amount': 1, 'unit': 'table_spoon'},
#  {'ingredientName': 'ผงปรุงรส', 'amount': 0.5, 'unit': 'table_spoon'},
#  {'ingredientName': 'น้ำเปล่า', 'amount': 2.125, 'unit': 'table_spoon'},
#  {'ingredientName': 'น้ำมัน', 'amount': 1, 'unit': 'table_spoon'}],'พริก')

In [ ]:
for ingredient_name in ingredient_dict.values():
  df[ingredient_name] = df['ingredients'].apply(lambda x : is_ingredients_in_list(x,ingredient_name))

In [ ]:
df

In [ ]:
df['น้ำปลา']

In [ ]:
import numpy as np

In [ ]:
min_by_tag = {}
mean_by_tag = {}

In [ ]:
def mean_and_min_by_tag(d_frame,tag):
  d_frame = d_frame[d_frame[tag] == 1]
  min_by_tag[tag] = {}
  mean_by_tag[tag] = {}
  for ingre_name in ingredient_dict.values():
    df_ingre = d_frame[d_frame[ingre_name] > 0.1]
    q_low = df_ingre[ingre_name].quantile(0.01)
    q_hi  = df_ingre[ingre_name].quantile(0.99)
    df_ingre_no_out_line = df_ingre[(df_ingre[ingre_name] < q_hi) & (df_ingre[ingre_name] > q_low)]
    mean = df_ingre_no_out_line[ingre_name].mean()
    minn = df_ingre_no_out_line[ingre_name].min()
    if not np.isnan(mean):
      mean_by_tag[tag][ingre_name] = float(mean)
    if not np.isnan(minn):
      min_by_tag[tag][ingre_name] = float(minn)


In [ ]:
# about 1.5 mins
for external_id in uniq_external_id:
  mean_and_min_by_tag(df,external_id)

In [ ]:
mean_by_tag

In [ ]:
min_by_tag

In [ ]:
# import json

# with open('../dictionary/mean_by_tag.json','w',encoding='utf8')as file:
#   json.dump(mean_by_tag,file,ensure_ascii=False)
# with open('../dictionary/min_by_tag.json','w',encoding='utf8')as file:
#   json.dump(min_by_tag,file,ensure_ascii=False)

In [ ]:
df

In [ ]:
def get_value_from_tag(ingre,tags,min = False):
  value = []
  if min:
    data = min_by_tag
  else:
    data = mean_by_tag
  for tag in tags:
    if ingre in data[tag]:
      value.append(data[tag][ingre])
  mean = np.array(value).mean()
  if np.isnan(mean):
    return 0
  else: return mean


In [ ]:
# get_value_from_tag('น้ำตาล',["Vegan-Recipe",'cc-pork'],min=True)

In [ ]:
df[['tags2']].sample(15)

In [ ]:
tags = df['tags2'].tolist()

In [ ]:
tags = [[t for t in x if t != 'main-dish-recipes'] for x in tags]

In [ ]:
tags

In [ ]:
for x in df:
  if x not in ingredient_dict.values():
    continue
  for y in range(len(df)):
    if df[x][y] == -1:
      df[x][y] = get_value_from_tag(x,tags[y],min=False)
      print(df[x][y])
    if df[x][y] == -2:
      df[x][y] = get_value_from_tag(x,tags[y],min=True)
      print(df[x][y])

In [ ]:
df

In [ ]:
df_model = df.drop(['url','allTimeScore','view','ingredients','photos','tags','tags2','is_has_amoungs'],axis=1)

In [ ]:
df_model

In [ ]:
df_model.to_csv('../data/data_for_model.csv',index=False)

In [ ]:
category_tag = {}

In [ ]:
for x in df['tags']:
  for tag in x:
    if tag['category'] not in category_tag:
      category_tag[tag['category']] = set()
    category_tag[tag['category']].add(tag['externalId'])


In [ ]:
cate_dict = {}
for k in category_tag:
  cate_dict[k] = list(category_tag[k])

In [ ]:
cate_dict

In [ ]:
import json

with open('../dictionary/tag_category.json','w',encoding='utf8')as file:
  json.dump(cate_dict,file,ensure_ascii=False)